In [ ]:
import csv
import math
import pandas as pd
import numpy as np

In [ ]:
def std_dev(distr, avg, num_samples):
    sum_of_squares = 0
    for k,v in distr.items():
        sum_of_squares += v*((k-avg)**2)
    variance = sum_of_squares/num_samples
    return math.sqrt(variance)

In [ ]:
training_df = pd.read_csv('training.csv')
training_df.drop('article_number', axis=1, inplace=True)
print(training_df)

In [ ]:
labels = training_df['topic']
label_distr = labels.value_counts()
print(label_distr)
print(label_distr.shape)
print(label_distr['IRRELEVANT'])
label_sum = label_distr.sum()
print(label_sum)
label_distr = pd.concat([pd.Series(data=[label_sum], index=['global']), label_distr])
print(label_distr)
print(label_distr['DEFENCE'])

In [ ]:
distr = {'global':{}}

for row in training_df.itertuples():
    sentence = row.article_words
    sentence = sentence.split(sep=',')
    words = len(sentence)
    if words not in distr['global']:
        distr['global'][words] = 0
    distr['global'][words] += 1
    #get label counts from dataframe
#     if 'count' not in distr['global']:
#         distr['global']['count'] = 0
#     distr['global']['count'] += 1
    if 'total' not in distr['global']:
        distr['global']['total'] = 0
    distr['global']['total'] += words
    #topic = row.topic.lower()
    topic = row.topic
    if topic not in distr:
        distr[topic] = {}
    if words not in distr[topic]:
        distr[topic][words] = 0
    distr[topic][words] += 1
    if 'total' not in distr[topic]:
        distr[topic]['total'] = 0
    distr[topic]['total'] += words

print(distr)

In [ ]:
##messy as hell
print('text length statistics')
for label, lengths in distr.items():
    num_samples = label_distr[label]
    avg = lengths['total']/num_samples
    del lengths['total'] #total should be separate...not in the distr data...classes might be easier
    filtered_lengths = [x for x in lengths.keys() if isinstance(x, int)]
    #print(filtered_lengths)
    deviation = std_dev(lengths, avg, num_samples)
    min_len = min(filtered_lengths)
    max_len = max(filtered_lengths)
    freqs = [v for k,v in lengths.items() if k != 'total']
    mode = max(freqs)
    modes = [k for k,v in lengths.items() if v == mode]
    coeff_variation = deviation/avg
    print(label, ":")
    print('\tavg: ', avg)
    print('\tstd_dev:', deviation)
    print('\tcoef_var:', coeff_variation)
    print('\tmin: ', min_len) 
    print('\tmax: ', max_len)
    print('\tmode: ', mode, modes)

In [ ]:
#corpus building
corpus = {}
for row in training_df.itertuples():
    words = row.article_words.split(',')
    for word in words:
        if word not in corpus:
            corpus[word] = {}
        if 'global' not in corpus[word]:
            corpus[word]['global'] = 0
        corpus[word]['global'] += 1
        if row.topic not in corpus[word]:
            corpus[word][row.topic] = 0
        corpus[word][row.topic] += 1
#print(corpus)

for word, word_dist in corpus.items():
    print(word)
    for topic, freq in word_dist.items():
        print('\t', topic, freq)

In [ ]:
##look into entropy and all that jazz and do some statistics on the corpus